# imports

In [ ]:
# --- Repo bootstrap (adds folder containing 'modules' to sys.path)
import os, sys

def _add_repo_root():
    cand = os.getcwd()
    root = None
    for _ in range(8):  # walk up to 8 parents
        if os.path.isdir(os.path.join(cand, "modules")):
            root = cand
            break
        cand = os.path.dirname(cand)

    if root is None:
        env = os.getenv("PROJECT_ROOT")
        if env and os.path.isdir(os.path.join(env, "modules")):
            root = env

    if root is None:
        raise SystemExit(
            "❌ Could not find repo root (folder containing 'modules'). "
            "Set PROJECT_ROOT env var or open the notebook from the repo."
        )

    if root not in sys.path:
        sys.path.insert(0, root)
    print("✓ Using repo root:", root)

_add_repo_root()


✓ Using repo root: c:\Users\felipeproenca\Documents\workspaces\personal\carbon-footprint


In [ ]:
# --- Load official BR ports dataset from repo
import os, json, importlib

# If you already have these imports above, it's fine to re-import
from modules.cabotage import load_ports, find_nearest_port
from modules.addressing import resolver
from modules.road.ors_client import ORSConfig, ORSClient

def _repo_root():
    cand = os.getcwd()
    for _ in range(8):
        if os.path.isdir(os.path.join(cand, "modules")):
            return cand
        cand = os.path.dirname(cand)
    env = os.getenv("PROJECT_ROOT")
    if env and os.path.isdir(os.path.join(env, "modules")):
        return env
    raise SystemExit("❌ Could not locate repo root (folder containing 'modules').")

ROOT = _repo_root()

# Preferred path (what you asked for) + a compatible fallback
CANDIDATES = [
    os.path.join(ROOT, "modules", "cabotage", "_data", "ports_br.json"),
    os.path.join(ROOT, "modules", "cabotage_data", "ports_br.json"),
]

ports_json = next((p for p in CANDIDATES if os.path.exists(p)), None)
if not ports_json:
    raise SystemExit(
        "❌ ports_br.json not found. Expected at:\n  - "
        + "\n  - ".join(CANDIDATES)
    )

ports = load_ports(ports_json)
print(f"✓ Loaded {len(ports)} ports from {ports_json}")


✓ Loaded 28 ports from c:\Users\felipeproenca\Documents\workspaces\personal\carbon-footprint\modules\cabotage\_data\ports_br.json


# Routes

In [ ]:
# --- Nearest-port sanity (gate-aware if present)
def jprint(obj): 
    print(json.dumps(obj, ensure_ascii=False, indent=2, sort_keys=True))

cfg = ORSConfig(default_country="BR")
ors = ORSClient(cfg)

def _show_best(origin_text: str):
    pt = resolver.resolve_point(origin_text, ors=ors)
    best = find_nearest_port(pt["lat"], pt["lon"], ports)
    gate = best.get("gate")
    print(f"\nOrigin: {origin_text}")
    print(f"Resolved: ({pt['lat']:.6f},{pt['lon']:.6f}) — {pt['label']}")
    print(f"Nearest port: {best['name']} ({best['city']}-{best['state']}) → {best['distance_km']:.2f} km")
    if gate:
        print(f"  via gate: {gate['label']} @ ({gate['lat']:.6f},{gate['lon']:.6f})")

for origin_text in [
    "São Paulo, SP",                    # → Santos (likely via Ponta da Praia)
    "Copacabana, Rio de Janeiro, RJ",  # → Rio de Janeiro
    "Curitiba, PR",                    # → Paranaguá
]:
    _show_best(origin_text)


[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='São Paulo, SP' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Copacabana, Rio de Janeiro, RJ' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Curitiba, PR' country=BR size=1



Origin: São Paulo, SP
Resolved: (-23.570533,-46.663713) — São Paulo, Brazil
Nearest port: Santos (Santos-SP) → 54.88 km
  via gate: Ponta da Praia @ (-23.960800,-46.333600)

Origin: Copacabana, Rio de Janeiro, RJ
Resolved: (-22.976478,-43.187679) — Copacabana, Rio de Janeiro, Brazil
Nearest port: Rio de Janeiro (Rio de Janeiro-RJ) → 8.68 km

Origin: Curitiba, PR
Resolved: (-25.459935,-49.280018) — Curitiba, PR, Brazil
Nearest port: Paranaguá (Paranaguá-PR) → 78.18 km


In [ ]:
# --- Batch smoke: common origins → nearest port printout
origins = [
      "São Paulo, SP"
    , "Campinas, SP"
    , "Sorocaba, SP"
    , "Uberlândia, MG"
    , "Belo Horizonte, MG"
    , "Vitória, ES"
    , "Brasília, DF"
    , "Goiânia, GO"
    , "Curitiba, PR"
    , "Florianópolis, SC"
    , "Porto Alegre, RS"
    , "Fortaleza, CE"
    , "Recife, PE"
    , "Salvador, BA"
]

for o in origins:
    pt  = resolver.resolve_point(o, ors=ors)
    best = find_nearest_port(pt["lat"], pt["lon"], ports)
    g = best.get("gate")
    via = f" via {g['label']}" if g else ""
    print(f"{o:28s} → {best['name']} ({best['city']}-{best['state']}){via}  [{best['distance_km']:.1f} km]")


[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='São Paulo, SP' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Campinas, SP' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Sorocaba, SP' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Uberlândia, MG' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Belo Horizonte, MG' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Vitória, ES' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Brasília, DF' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Goiânia, GO' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Curitiba, PR' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE text='Florianópolis, SC' country=BR size=1
[17:20:17] INFO cabosupernet.road.ors_client | GEOCODE tex

São Paulo, SP                → Santos (Santos-SP) via Ponta da Praia  [54.9 km]
Campinas, SP                 → Santos (Santos-SP) via Ponta da Praia  [141.7 km]
Sorocaba, SP                 → Santos (Santos-SP) via Ponta da Praia  [125.9 km]
Uberlândia, MG               → Santos (Santos-SP) via Ponta da Praia  [595.1 km]
Belo Horizonte, MG           → Itaguaí (Sepetiba) (Itaguaí-RJ)  [336.0 km]
Vitória, ES                  → Vitória / TVV (Vila Velha) (Vila Velha-ES)  [3.9 km]
Brasília, DF                 → Angra dos Reis (Angra dos Reis-RJ)  [884.6 km]
Goiânia, GO                  → Santos (Santos-SP) via Ponta da Praia  [867.3 km]
Curitiba, PR                 → Paranaguá (Paranaguá-PR)  [78.2 km]
Florianópolis, SC            → Imbituba (Imbituba-SC)  [72.6 km]
Porto Alegre, RS             → Rio Grande (Rio Grande-RS)  [231.9 km]
Fortaleza, CE                → Fortaleza (Mucuripe) (Fortaleza-CE)  [7.8 km]
Recife, PE                   → Recife (Recife-PE)  [5.6 km]
Salvador, BA        

# Consumption

In [ ]:
# --- Test: cabotage router with SeaMatrix (matrix-preferred, haversine fallback)

from pathlib import Path
import json
from pprint import pprint

# 1) Imports from your project
from modules.cabotage.router import (
      find_cabotage_route
    , compute_port_to_port_sea_km
    , DEFAULTS
)
from modules.cabotage import router as _router_mod  # to optionally inject label aliases
from modules.cabotage.sea_matrix import SeaMatrix

# 2) Load the sea-distance matrix (your JSON)
SEA_MATRIX_PATH = Path("../modules/cabotage/_data/sea_matrix.json")
assert SEA_MATRIX_PATH.exists(), f"Matrix file not found: {SEA_MATRIX_PATH}"

with SEA_MATRIX_PATH.open("r", encoding="utf-8") as f:
    matrix_payload = json.load(f)

sea_matrix = SeaMatrix.from_json_dict(matrix_payload)
print(f"✅ SeaMatrix loaded with {sea_matrix.size()} labeled ports.")

# 2.1) (Optional) Inject aliases so router/matrix labels match
_aliases = {
    # matrix label ← project/router/ports_index label
    "Suape": "Ipojuca",
    "Fortaleza (Mucuripe)": "Fortaleza",
    "Pecém": "São Gonçalo do Amarante",
    "Vitória / TVV (Vila Velha)": "Vila Velha",
    "Santos": "Guarujá",  # sometimes ports_index uses Guarujá for the complex
}
if hasattr(_router_mod, "PORT_LABEL_ALIASES"):
    _router_mod.PORT_LABEL_ALIASES.update(_aliases)  # no-op if not present
else:
    # If your router doesn't expose a dict, no problem; lookups will still fall back.
    pass

# 3) ORS client (real if available; otherwise a richer fallback with cfg+geocode)
def _norm_key(s: str) -> str:
    return " ".join(s.casefold().split())

try:
    from modules.road.ors_client import ORSClient
    class _Cfg: default_country = "BR"
    ors = ORSClient(cfg=_Cfg())  # tweak if your ORSClient expects different args
    print("✅ Using real ORSClient.")
except Exception as e:
    print(f"⚠️ Real ORSClient not available ({e}). Using a fallback with geocode().")
    import math
    class _MockORS:
        class _Cfg:
            default_country = "BR"
        def __init__(self):
            self.cfg = self._Cfg()
            self._gaz = {
                _norm_key("São Paulo, SP"): {"lat": -23.55052, "lon": -46.63331, "label": "São Paulo, SP"},
                _norm_key("Recife, PE"):    {"lat":  -8.04756, "lon": -34.87700, "label": "Recife, PE"},
                _norm_key("Campinas, SP"):  {"lat": -22.90994, "lon": -47.06263, "label": "Campinas, SP"},
                _norm_key("Fortaleza, CE"): {"lat":  -3.73186, "lon": -38.52667, "label": "Fortaleza, CE"},
            }
        def geocode(self, text: str, country: str = "BR", size: int = 1):
            hit = self._gaz.get(_norm_key(text))
            if hit:
                return [hit][:size]
            # last resort so tests don't crash
            return [{"lat": -15.793889, "lon": -47.882778, "label": text}][:size]
        @staticmethod
        def _haversine(lat1, lon1, lat2, lon2):
            R = 6371.0088
            import math as _m
            a1 = _m.radians(lat1); b1 = _m.radians(lon1)
            a2 = _m.radians(lat2); b2 = _m.radians(lon2)
            da = a2 - a1; db = b2 - b1
            s  = (_m.sin(da/2)**2 + _m.cos(a1) * _m.cos(a2) * _m.sin(db/2)**2)
            return 2 * R * _m.atan2(_m.sqrt(s), _m.sqrt(1 - s))
        def route_road(self, o: dict, d: dict):
            km  = self._haversine(o["lat"], o["lon"], d["lat"], d["lon"]) * 1.05
            sec = (km / 50.0) * 3600.0
            return {"origin": o, "destination": d, "distance_m": km * 1000.0, "duration_s": sec}
    ors = _MockORS()

# 4) Helper to print a compact summary
def show_summary(tag: str, route: dict):
    print("\n" + "="*80)
    print(f"[{tag}]   {route['origin']['label']}  →  {route['destination']['label']}")
    po, pd = route["ports_used"]
    print(f"Ports used: {po['name']} → {pd['name']}")
    sea_leg = next(l for l in route["legs"] if l["mode"] == "sea")
    src = sea_leg.get("extras", {}).get("distance_source", "?")
    print(f"Sea distance (km): {sea_leg['distance_km']:.1f}  (source={src})")
    print(f"Sea hours: {sea_leg['hours']:.2f} (incl. handling)")
    print(f"Totals → km: {route['totals']['distance_km']:.1f} | "
          f"hours: {route['totals']['hours']:.2f} | "
          f"cost BRL: {route['totals']['cost_brl']:.2f} | "
          f"CO₂ t: {route['totals']['co2eq_t']:.4f}")

# 5) Scenarios that should hit the matrix
tests = [
      ("São Paulo, SP",  "Recife, PE")       # Santos ↔ Recife (matrix)
    , ("Campinas, SP",   "Fortaleza, CE")    # Santos/São Sebastião ↔ Fortaleza/Pecém (matrix)
]

# Optional overrides to defaults
overrides = dict(DEFAULTS)

# 6) Run tests
for i, (orig, dest) in enumerate(tests, 1):
    route = find_cabotage_route(
          origin=orig
        , destination=dest
        , ors=ors
        , sea_matrix=sea_matrix
        , **overrides
    )
    # Basic sanity checks
    assert len(route["legs"]) == 3, "Expected 3 legs (road, sea, road)."
    sea_leg = next(l for l in route["legs"] if l["mode"] == "sea")
    assert sea_leg["distance_km"] >= 0.0
    assert sea_leg.get("extras", {}).get("distance_source") in {"matrix", "haversine"}

    # Print summary & a quick matrix-vs-computed check
    show_summary(f"Test #{i}", route)

    # Verify matrix query directly for the chosen ports (transparency)
    pf, pt = route["ports_used"]
    km_mx = compute_port_to_port_sea_km(
          {"name": pf["name"], "lat": pf["lat"], "lon": pf["lon"]}  # canonical dicts
        , {"name": pt["name"], "lat": pt["lat"], "lon": pt["lon"]}
        , sea_matrix=sea_matrix
    )
    print(f"Matrix check {pf['name']} → {pt['name']}: {km_mx:.1f} km")

    # To enforce matrix usage only, uncomment the assertion below:
    # assert sea_leg["extras"]["distance_source"] == "matrix", "SeaMatrix pair missing; fell back to haversine."

print("\n✅ Done. If you saw 'source=matrix' above, the SeaMatrix was used successfully.")


✅ SeaMatrix loaded with 28 labeled ports.
⚠️ Real ORSClient not available ('_Cfg' object has no attribute 'cache_path'). Using a fallback with geocode().


AttributeError: '_MockORS' object has no attribute 'geocode_text'

In [ ]:
# ✅ Basic test for cabotage accounting

from modules.cabotage import *

# 0) (Optional) sanity: ports loader points to _data/ports_br.json
ports = load_ports(path=get_ports_path())
print(f"Loaded {len(ports)} ports")

# 1) Build a tiny 2-leg loop and 2 shipments
legs = [
      Leg(id="Santos→Suape",     distance_km=2000.0)
    , Leg(id="Suape→Fortaleza",  distance_km=800.0)
]
shipments = [
      Shipment(id="S1", weight_t=20.0, on_legs=[0, 1])   # onboard both legs
    , Shipment(id="S2", weight_t=10.0, on_legs=[0])      # drops at Suape
]

# 2) Load K from k.json (trimmed mean recommended)
k_repo = load_k_entries(path=get_k_path())
K = choose_K(k_repo, mode="trimmed")
print(f"K = {K:.6f} kg fuel / t·km")

# 3) Allocate fuel per leg, then to shipments
fuel_by_leg_kg, fuel_by_ship_kg, fuel_total_kg = allocate_fuel_by_leg_and_shipment(
      K_kg_per_tkm=K
    , legs=legs
    , shipments=shipments
)
print("fuel_by_leg_kg  →", fuel_by_leg_kg)
print("fuel_by_ship_kg →", fuel_by_ship_kg)
print("fuel_total_kg   →", fuel_total_kg)

# 4) Cost & emissions (plug your factors and bunker price)
price_per_tonne_brl = 3200.0
ef_ttw_mgo = {"CO2": 3206.0, "CH4": 0.0, "N2O": 0.0}
gwp100_ar6 = {"CH4": 29.8, "N2O": 273.0}

report = allocate_costs_emissions(
      fuel_by_shipment_kg=fuel_by_ship_kg
    , price_per_tonne=price_per_tonne_brl
    , ef_ttw_per_tonne_fuel=ef_ttw_mgo
    , gwp100=gwp100_ar6
)
report


Loaded 28 ports
K = 0.002775 kg fuel / t·km
fuel_by_leg_kg  → {0: 166.512, 1: 44.403200000000005}
fuel_by_ship_kg → {'S1': 155.4112, 'S2': 55.504}
fuel_total_kg   → 210.9152


{'S1': {'fuel_kg': 155.4112,
  'fuel_t': 0.1554112,
  'cost': 497.31584,
  'CO2': 498.2483072,
  'CH4': 0.0,
  'N2O': 0.0,
  'CO2e': 498.2483072},
 'S2': {'fuel_kg': 55.504,
  'fuel_t': 0.055504,
  'cost': 177.6128,
  'CO2': 177.945824,
  'CH4': 0.0,
  'N2O': 0.0,
  'CO2e': 177.945824}}

In [ ]:
# ✅ Port-operations fuel only (no sea leg fuel, no costs/emissions)

# Imports (pull the allocator straight from accounting to avoid __init__ export dependencies)
from modules.cabotage.accounting import allocate_port_fuel_to_shipments
from modules.cabotage import Leg, Shipment

# 0) Example legs & shipments (replace with your actual objects)
legs = [
      Leg(id="Santos→Suape",     distance_km=2000.0)
    , Leg(id="Suape→Fortaleza",  distance_km=800.0)
]
shipments = [
      Shipment(id="S1", weight_t=20.0, on_legs=[0, 1])   # load at Santos, discharge at Fortaleza
    , Shipment(id="S2", weight_t=10.0, on_legs=[0])      # load at Santos, discharge at Suape
]

# 1) Helper: parse origin/destination port names from Leg.id (supports "→" or "->")
def _split_ports(leg_id: str) -> tuple[str, str]:
    if "→" in leg_id:
        a, b = leg_id.split("→", 1)
    elif "->" in leg_id:
        a, b = leg_id.split("->", 1)
    else:
        # fallback: try common hyphen arrow
        parts = [p.strip() for p in leg_id.replace("—", "->").replace("-", "->").split("->") if p.strip()]
        if len(parts) == 2:
            a, b = parts
        else:
            raise ValueError(f"Cannot parse origin/destination from leg id: {leg_id}")
    return a.strip(), b.strip()

# 2) Build a list of (origin, destination) for each leg index
od_by_leg = {}
for i, leg in enumerate(legs):
    o, d = _split_ports(leg.id)
    od_by_leg[i] = (o, d)

# 3) Infer handled masses per port (load at first-leg origin; discharge at last-leg destination)
#    handled_by_port = {port_name: {shipment_id: tonnes_handled_here, ...}, ...}
handled_by_port: dict[str, dict[str, float]] = {}

for s in shipments:
    if not s.on_legs:
        continue
    first_leg = min(s.on_legs)
    last_leg  = max(s.on_legs)

    o_port, _ = od_by_leg[first_leg]
    _, d_port = od_by_leg[last_leg]

    # load at origin
    handled_by_port.setdefault(o_port, {})
    handled_by_port[o_port][s.id] = handled_by_port[o_port].get(s.id, 0.0) + s.weight_t

    # discharge at destination
    handled_by_port.setdefault(d_port, {})
    handled_by_port[d_port][s.id] = handled_by_port[d_port].get(s.id, 0.0) + s.weight_t

# 4) Apply a single constant K_port (kg fuel per tonne handled) per port call
K_port = 0.48  # tweak per terminal if you have better data (≈0.40–0.50 kg/t typical diesel ops)

fuel_by_port_by_ship_kg: dict[str, dict[str, float]] = {}
fuel_by_port_total_kg: dict[str, float] = {}
fuel_total_kg = 0.0

for port, handled_by_ship in handled_by_port.items():
    f_by_ship = allocate_port_fuel_to_shipments(
          handled_mass_by_shipment_t = handled_by_ship
        , K_port_kg_per_t            = K_port
    )
    fuel_by_port_by_ship_kg[port] = f_by_ship
    port_total = sum(f_by_ship.values())
    fuel_by_port_total_kg[port] = port_total
    fuel_total_kg += port_total

# 5) Aggregate per-shipment totals across all ports
fuel_by_shipment_total_kg: dict[str, float] = {}
for port, f_by_ship in fuel_by_port_by_ship_kg.items():
    for sid, fkg in f_by_ship.items():
        fuel_by_shipment_total_kg[sid] = fuel_by_shipment_total_kg.get(sid, 0.0) + fkg

# 6) Pretty print
print("Handled tonnes per port:")
for port, d in handled_by_port.items():
    print(f"  {port}: {d}")

print("\nPort-ops fuel by port (kg):")
for port, fkg in fuel_by_port_total_kg.items():
    print(f"  {port}: {fkg:.2f} kg")

print("\nPort-ops fuel by port × shipment (kg):")
for port, f_by_ship in fuel_by_port_by_ship_kg.items():
    print(f"  {port}: { {k: round(v, 3) for k, v in f_by_ship.items()} }")

print("\nPort-ops fuel by shipment (kg):")
for sid, fkg in fuel_by_shipment_total_kg.items():
    print(f"  {sid}: {fkg:.2f} kg")

print(f"\nTOTAL port-ops fuel: {fuel_total_kg:.2f} kg")

# If you want the raw dicts as the cell output:
{
      "handled_by_port": handled_by_port
    , "fuel_by_port_total_kg": fuel_by_port_total_kg
    , "fuel_by_port_by_ship_kg": fuel_by_port_by_ship_kg
    , "fuel_by_shipment_total_kg": fuel_by_shipment_total_kg
    , "fuel_total_kg": fuel_total_kg
}


Handled tonnes per port:
  Santos: {'S1': 20.0, 'S2': 10.0}
  Fortaleza: {'S1': 20.0}
  Suape: {'S2': 10.0}

Port-ops fuel by port (kg):
  Santos: 14.40 kg
  Fortaleza: 9.60 kg
  Suape: 4.80 kg

Port-ops fuel by port × shipment (kg):
  Santos: {'S1': 9.6, 'S2': 4.8}
  Fortaleza: {'S1': 9.6}
  Suape: {'S2': 4.8}

Port-ops fuel by shipment (kg):
  S1: 19.20 kg
  S2: 9.60 kg

TOTAL port-ops fuel: 28.80 kg


{'handled_by_port': {'Santos': {'S1': 20.0, 'S2': 10.0},
  'Fortaleza': {'S1': 20.0},
  'Suape': {'S2': 10.0}},
 'fuel_by_port_total_kg': {'Santos': 14.399999999999999,
  'Fortaleza': 9.6,
  'Suape': 4.8},
 'fuel_by_port_by_ship_kg': {'Santos': {'S1': 9.6, 'S2': 4.8},
  'Fortaleza': {'S1': 9.6},
  'Suape': {'S2': 4.8}},
 'fuel_by_shipment_total_kg': {'S1': 19.2, 'S2': 9.6},
 'fuel_total_kg': 28.8}

In [ ]:
# ✅ Port-ops (hotel-at-berth) fuel using modules/cabotage/_data/hotel.json

import os, sys, json, unicodedata
from statistics import mean

# Reuse your simple classes
from modules.cabotage import Leg, Shipment
from modules.cabotage.accounting import allocate_port_fuel_to_shipments

# ────────────────────────────────────────────────────────────────────────────────
# Locate repo root (folder containing 'modules')
# ────────────────────────────────────────────────────────────────────────────────
def _repo_root():
    cand = os.getcwd()
    for _ in range(8):
        if os.path.isdir(os.path.join(cand, "modules")):
            return cand
        cand = os.path.dirname(cand)
    env = os.getenv("PROJECT_ROOT")
    if env and os.path.isdir(os.path.join(env, "modules")):
        return env
    raise SystemExit("❌ Could not locate repo root (folder containing 'modules').")

ROOT = _repo_root()
HOTEL_JSON = os.path.join(ROOT, "modules", "cabotage", "_data", "hotel.json")

# ────────────────────────────────────────────────────────────────────────────────
# Load per-city hotel factors (kg_fuel_per_t)
# ────────────────────────────────────────────────────────────────────────────────
with open(HOTEL_JSON, "r", encoding="utf-8") as f:
    hotel_payload = json.load(f)

entries = hotel_payload.get("entries", [])
city_factor = { e["city"]: float(e["kg_fuel_per_t"]) for e in entries }

# Fallback = trimmed-mean (drop 10% tails if enough samples)
vals = sorted([v for v in city_factor.values() if v > 0])
if len(vals) >= 20:
    cut = max(1, int(0.10 * len(vals)))
    vals_mm = vals[cut:-cut]
    fallback_K = mean(vals_mm) if vals_mm else mean(vals)
else:
    fallback_K = mean(vals) if vals else 0.75  # conservative default

# Common aliases: leg labels → municipality in ANTAQ
ALIASES = {
      "Suape": "Ipojuca"
    , "Pecém": "São Gonçalo do Amarante"
    , "Porto de Suape": "Ipojuca"
    , "Porto do Açu": "São João da Barra"
    , "Porto de Itaguaí": "Itaguaí"
    , "Porto de Santos": "Santos"  # sometimes "Guarujá", keep as-is in legs if desired
}

def _norm(s: str) -> str:
    s = (s or "").strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return s

def k_for_port_label(port_label: str) -> tuple[float, str]:
    """
    Return (K_port_kg_per_t, resolved_city) for a leg's port label.
    Tries: exact city match → alias → accent-insensitive scan → fallback_K.
    """
    # 1) exact
    if port_label in city_factor:
        return city_factor[port_label], port_label
    # 2) alias
    alias = ALIASES.get(port_label)
    if alias and alias in city_factor:
        return city_factor[alias], alias
    # 3) accent-insensitive scan
    nlabel = _norm(port_label)
    for city, k in city_factor.items():
        if _norm(city) == nlabel:
            return k, city
    # 4) fallback
    return fallback_K, f"{port_label} (fallback)"

# ────────────────────────────────────────────────────────────────────────────────
# Tiny demo voyage (same as before)
# ────────────────────────────────────────────────────────────────────────────────
legs = [
      Leg(id="Santos→Suape",     distance_km=2000.0)
    , Leg(id="Suape→Fortaleza",  distance_km=800.0)
]
shipments = [
      Shipment(id="S1", weight_t=20.0, on_legs=[0, 1])   # load at Santos, discharge at Fortaleza
    , Shipment(id="S2", weight_t=10.0, on_legs=[0])      # load at Santos, discharge at Suape
]

# Parse "A→B" port names from leg id
def _split_ports(leg_id: str) -> tuple[str, str]:
    if "→" in leg_id:
        a, b = leg_id.split("→", 1)
    elif "->" in leg_id:
        a, b = leg_id.split("->", 1)
    else:
        parts = [p.strip() for p in leg_id.replace("—", "->").replace("-", "->").split("->") if p.strip()]
        if len(parts) != 2:
            raise ValueError(f"Cannot parse origin/destination from leg id: {leg_id}")
        a, b = parts
    return a.strip(), b.strip()

# Build leg → (origin, destination) map
od_by_leg = {}
for i, leg in enumerate(legs):
    o, d = _split_ports(leg.id)
    od_by_leg[i] = (o, d)

# Infer tonnes handled per port (load at first-leg origin; discharge at last-leg destination)
handled_by_port: dict[str, dict[str, float]] = {}
for s in shipments:
    if not s.on_legs:
        continue
    first_leg = min(s.on_legs)
    last_leg  = max(s.on_legs)
    o_port, _ = od_by_leg[first_leg]
    _, d_port = od_by_leg[last_leg]

    handled_by_port.setdefault(o_port, {})
    handled_by_port[o_port][s.id] = handled_by_port[o_port].get(s.id, 0.0) + s.weight_t

    handled_by_port.setdefault(d_port, {})
    handled_by_port[d_port][s.id] = handled_by_port[d_port].get(s.id, 0.0) + s.weight_t

# Allocate hotel fuel using city-specific K_port per port label
fuel_by_port_by_ship_kg: dict[str, dict[str, float]] = {}
fuel_by_port_total_kg: dict[str, float] = {}
fuel_by_shipment_total_kg: dict[str, float] = {}
fuel_total_kg = 0.0

print("Hotel factors (kg fuel per tonne) by port label:")
for port_label in handled_by_port.keys():
    Kp, resolved_city = k_for_port_label(port_label)
    print(f"  {port_label:20s} → {resolved_city:28s} : K_port = {Kp:.6f} kg/t")

print("\nHandled tonnes per port:")
for port, d in handled_by_port.items():
    print(f"  {port}: {d}")

for port_label, handled_by_ship in handled_by_port.items():
    Kp, resolved_city = k_for_port_label(port_label)
    f_by_ship = allocate_port_fuel_to_shipments(
          handled_mass_by_shipment_t = handled_by_ship
        , K_port_kg_per_t            = Kp
    )
    fuel_by_port_by_ship_kg[port_label] = f_by_ship
    port_total = sum(f_by_ship.values())
    fuel_by_port_total_kg[port_label] = port_total
    fuel_total_kg += port_total

    for sid, fkg in f_by_ship.items():
        fuel_by_shipment_total_kg[sid] = fuel_by_shipment_total_kg.get(sid, 0.0) + fkg

print("\nPort-ops (hotel) fuel by port × shipment (kg):")
for port, f_by_ship in fuel_by_port_by_ship_kg.items():
    pretty = {k: round(v, 3) for k, v in f_by_ship.items()}
    print(f"  {port}: {pretty}")

print("\nPort-ops (hotel) fuel by port (kg):")
for port, fkg in fuel_by_port_total_kg.items():
    print(f"  {port}: {fkg:.2f} kg")

print("\nPort-ops (hotel) fuel by shipment (kg):")
for sid, fkg in fuel_by_shipment_total_kg.items():
    print(f"  {sid}: {fkg:.2f} kg")

print(f"\nTOTAL port-ops (hotel) fuel: {fuel_total_kg:.2f} kg")

# Structured output (dict) if you want the cell to return something
{
      "hotel_json": HOTEL_JSON
    , "fallback_K_kg_per_t": round(fallback_K, 6)
    , "handled_by_port": handled_by_port
    , "fuel_by_port_total_kg": fuel_by_port_total_kg
    , "fuel_by_port_by_ship_kg": fuel_by_port_by_ship_kg
    , "fuel_by_shipment_total_kg": fuel_by_shipment_total_kg
    , "fuel_total_kg": fuel_total_kg
}


Hotel factors (kg fuel per tonne) by port label:
  Santos               → Santos                       : K_port = 0.874877 kg/t
  Fortaleza            → Fortaleza                    : K_port = 0.850601 kg/t
  Suape                → Ipojuca                      : K_port = 0.627424 kg/t

Handled tonnes per port:
  Santos: {'S1': 20.0, 'S2': 10.0}
  Fortaleza: {'S1': 20.0}
  Suape: {'S2': 10.0}

Port-ops (hotel) fuel by port × shipment (kg):
  Santos: {'S1': 17.498, 'S2': 8.749}
  Fortaleza: {'S1': 17.012}
  Suape: {'S2': 6.274}

Port-ops (hotel) fuel by port (kg):
  Santos: 26.25 kg
  Fortaleza: 17.01 kg
  Suape: 6.27 kg

Port-ops (hotel) fuel by shipment (kg):
  S1: 34.51 kg
  S2: 15.02 kg

TOTAL port-ops (hotel) fuel: 49.53 kg


{'hotel_json': 'c:\\Users\\felipeproenca\\Documents\\workspaces\\personal\\carbon-footprint\\modules\\cabotage\\_data\\hotel.json',
 'fallback_K_kg_per_t': 0.882903,
 'handled_by_port': {'Santos': {'S1': 20.0, 'S2': 10.0},
  'Fortaleza': {'S1': 20.0},
  'Suape': {'S2': 10.0}},
 'fuel_by_port_total_kg': {'Santos': 26.24631,
  'Fortaleza': 17.01202,
  'Suape': 6.27424},
 'fuel_by_port_by_ship_kg': {'Santos': {'S1': 17.49754, 'S2': 8.74877},
  'Fortaleza': {'S1': 17.01202},
  'Suape': {'S2': 6.27424}},
 'fuel_by_shipment_total_kg': {'S1': 34.50956, 'S2': 15.02301},
 'fuel_total_kg': 49.53257}